In [1]:
# merge data
# data transformation
# store preprocessing data
import numpy as np
import pandas as pd
import os

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from joblib import dump,load

In [4]:
!dir data

 驱动器 C 中的卷没有标签。
 卷的序列号是 0006-FDC5

 C:\Users\dell\dockerdata\乘用车细分市场销量预测\data 的目录

2019/09/24  17:14    <DIR>          .
2019/09/24  17:14    <DIR>          ..
2019/09/11  13:30           237,991 evaluation_public.csv
2019/09/19  14:39            51,762 mean_predicting_data.csv
2019/09/25  17:53            51,255 rf_prediction_default.csv
2019/09/11  13:31            51,712 submit_example.csv
2019/09/20  16:14            51,711 svr_predicting_sales.csv
2019/08/23  17:36           369,997 Train.zip
2019/09/12  15:33         1,545,277 train_sales_data.csv
2019/09/12  15:33         1,399,503 train_search_data.csv
2019/09/12  15:33            47,975 train_user_reply_data.csv
               9 个文件      3,807,183 字节
               2 个目录 47,271,370,752 可用字节


In [5]:
train_sales_data=pd.read_csv('data/train_sales_data.csv')

In [6]:
model_bodytype=train_sales_data[['model','bodyType']].drop_duplicates()
#model_bodytype.info()

In [7]:
evaluation_public_data=pd.read_csv('data/evaluation_public.csv')
evaluation_public_data.head()

,id,province,adcode,model,regYear,regMonth,forecastVolum
0,1,上海,310000,3c974920a76ac9c1,2018,1,NaN
1,2,云南,530000,3c974920a76ac9c1,2018,1,NaN
2,3,内蒙古,150000,3c974920a76ac9c1,2018,1,NaN
3,4,北京,110000,3c974920a76ac9c1,2018,1,NaN
4,5,四川,510000,3c974920a76ac9c1,2018,1,NaN


In [8]:
train_sales_data_2017=train_sales_data[train_sales_data['regYear']==2017]
train_sales_data_2016=train_sales_data[train_sales_data['regYear']==2016]

In [9]:
train_sales_data_2017.head(1)

,province,adcode,model,bodyType,regYear,regMonth,salesVolume
15840,上海,310000,3c974920a76ac9c1,SUV,2017,1,305


In [10]:
# merge data on the month of year before
def merge_year_before(df,spdf,cols):
    '''
    input:
        df(pd.DataFrame)-the left merging data df
        psdf(pd.DataFrame)-the right merging data df
    putput:
        mergeddf-the left merged data df
    '''
    onlist=['province','model','regMonth']
    mergeddf=pd.merge(df,spdf,on=onlist,how='left',suffixes=('','_year_before'))
    #cols=['province','adcode','model','bodyType','regYear','regMonth','salesVolume_year_before','salesVolume']
    return mergeddf[cols]

In [11]:
cols=['province','adcode','model','bodyType','regYear','regMonth','salesVolume_year_before','salesVolume']
train_sales_year_before_data=merge_year_before(train_sales_data_2017,train_sales_data_2016,cols)
train_sales_year_before_data.head()

,province,adcode,model,bodyType,regYear,regMonth,salesVolume_year_before,salesVolume
0,上海,310000,3c974920a76ac9c1,SUV,2017,1,292,305
1,云南,530000,3c974920a76ac9c1,SUV,2017,1,466,434
2,内蒙古,150000,3c974920a76ac9c1,SUV,2017,1,257,180
3,北京,110000,3c974920a76ac9c1,SUV,2017,1,408,468
4,四川,510000,3c974920a76ac9c1,SUV,2017,1,610,500


In [12]:
cols=['id','province','adcode','model','bodyType','regYear','regMonth','salesVolume']
evaluation_public_year_before_data=merge_year_before(evaluation_public_data,train_sales_data_2017,cols)
evaluation_public_year_before_data.columns=['id','province','adcode','model','bodyType','regYear',\
                                            'regMonth','salesVolume_year_before']
evaluation_public_year_before_data.head()

,id,province,adcode,model,bodyType,regYear,regMonth,salesVolume_year_before
0,1,上海,310000,3c974920a76ac9c1,SUV,2018,1,305
1,2,云南,530000,3c974920a76ac9c1,SUV,2018,1,434
2,3,内蒙古,150000,3c974920a76ac9c1,SUV,2018,1,180
3,4,北京,110000,3c974920a76ac9c1,SUV,2018,1,468
4,5,四川,510000,3c974920a76ac9c1,SUV,2018,1,500


In [13]:
# 生成 datetime 列
def generate_date(df):
    f=lambda x:str(x) if x >9 else '0'+str(x)
    df['date']=df['regYear'].map(lambda x:str(x))+'-'+df['regMonth'].map(f)
    df['date']=pd.to_datetime(df['date'],format='%Y-%m')
    return df

In [14]:
# 日期偏移 month
def month_offset_date(df,n=1):
    from pandas.tseries.offsets import DateOffset
    df['offdate']=df['date']+DateOffset(months=n)
    return df

In [15]:
# 日期偏移 year
def year_offset_date(df,n=1):
    from pandas.tseries.offsets import DateOffset
    df['offdate']=df['date']+DateOffset(years=n)
    return df

In [16]:
#merge data month before
def merge_month_before(df,spdf,cols):
    '''
    input:
        df(pd.DataFrame)--left merging data df
        spdf(pd.DataFrame)--right merging data df
    output:
        mergedf(pd.DataFrame)--merged data df
    '''
    
    df=generate_date(df)
    spdf=generate_date(spdf)
    spdf=month_offset_date(spdf)
    mergedf=pd.merge(df,spdf,how='left',left_on=['province','model','date'],right_on=['province','model','offdate'],\
                    suffixes=('','_month_before'))
    #cols=['id','province','adcode','model','regYear','regMonth','salesVolume_month_before']
    #return mergedf
    return mergedf[cols]

In [17]:
cols=['province','adcode','model','bodyType','regYear','regMonth','salesVolume_month_before']
train_sales_month_before=merge_month_before(train_sales_data_2017,train_sales_data,cols)
train_sales_month_before.head(2)

,province,adcode,model,bodyType,regYear,regMonth,salesVolume_month_before
0,上海,310000,3c974920a76ac9c1,SUV,2017,1,374
1,云南,530000,3c974920a76ac9c1,SUV,2017,1,456


In [18]:
train_sales_month_before.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15840 entries, 0 to 15839
Data columns (total 7 columns):
province                    15840 non-null object
adcode                      15840 non-null int64
model                       15840 non-null object
bodyType                    15840 non-null object
regYear                     15840 non-null int64
regMonth                    15840 non-null int64
salesVolume_month_before    15840 non-null int64
dtypes: int64(4), object(3)
memory usage: 990.0+ KB


In [19]:
train_data=pd.merge(train_sales_month_before,train_sales_year_before_data,\
                    on=['province','adcode','model','bodyType','regYear','regMonth'],\
                   how='left',suffixes=('','_right'))
train_data.head()

,province,adcode,model,bodyType,regYear,regMonth,salesVolume_month_before,salesVolume_year_before,salesVolume
0,上海,310000,3c974920a76ac9c1,SUV,2017,1,374,292,305
1,云南,530000,3c974920a76ac9c1,SUV,2017,1,456,466,434
2,内蒙古,150000,3c974920a76ac9c1,SUV,2017,1,219,257,180
3,北京,110000,3c974920a76ac9c1,SUV,2017,1,563,408,468
4,四川,510000,3c974920a76ac9c1,SUV,2017,1,570,610,500


In [20]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15840 entries, 0 to 15839
Data columns (total 9 columns):
province                    15840 non-null object
adcode                      15840 non-null int64
model                       15840 non-null object
bodyType                    15840 non-null object
regYear                     15840 non-null int64
regMonth                    15840 non-null int64
salesVolume_month_before    15840 non-null int64
salesVolume_year_before     15840 non-null int64
salesVolume                 15840 non-null int64
dtypes: int64(6), object(3)
memory usage: 1.2+ MB


In [21]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
ohe=OneHotEncoder()
shuffle(train_data)
train_cate_ohe=ohe.fit_transform(train_data[['province','model','bodyType']])
train_x=np.concatenate((train_cate_ohe.toarray(),train_data[['regMonth',\
                        'salesVolume_month_before','salesVolume_year_before']].values),
                 axis=1)
train_y=train_data['salesVolume'].values.ravel()

In [42]:
# 数据保存
np.savez('data/month_year_train.npz',x=train_x,y=train_y)

In [25]:
reg.score(eva_x,eva_y)

0.8929763913764569

In [26]:
eva_y_pred=reg.predict(eva_x)
mean_metrics(eva_y_pred,eva_y)

0.563783756693436

In [27]:
# Linear_model L2 Ridge
ridge=linear_model.Ridge(alpha=1.0)
ridge.fit(x,y)
ridge.score(x,y)

0.9059903547679712

In [28]:
ridge.score(eva_x,eva_y)

0.8929630599610981

In [186]:
#SVR 奇差无比
from sklearn.svm import SVR
svm_reg=SVR()
svm_reg.fit(x,y)
svm_reg.score(x,y)

-0.086177011335687

In [29]:
#RF 随机森林
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
rf_reg=RandomForestRegressor()
rf_reg.fit(x,y)
cross_val_score(rf_reg,x,y,cv=5)

array([0.92659167, 0.92462327, 0.92398945, 0.93312354, 0.93603297])

In [35]:
rf_reg

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [30]:
rf_reg.score(eva_x,eva_y)

0.9374506067146188

In [48]:
# 计算 平均 metric
def mean_metrics(y_pred,y):
    n_samples=y_pred.shape[0]
    y_mean=np.mean(y)
    return (1-((((y-y_pred)**2).sum()/n_samples)**0.5)/y_mean)

#处理test数据

In [33]:
eva_y_pred=rf_reg.predict(eva_x)
mean_metrics(eva_y_pred,eva_y)

0.6665170851708622

In [34]:
y_pred=rf_reg.predict(x)
mean_metrics(y_pred,y)

0.8617453632901322

In [1]:
import numpy as np
import pandas as pd
df = pd.DataFrame( {'a':['A','A','B','B','B','C'], 'b':[1,2,5,5,4,6]})

In [15]:
df.groupby('a')['b'].agg([list,'mean']).reset_index()

,a,list,mean
0,A,"[1, 2]",1.500000
1,B,"[5, 5, 4]",4.666667
2,C,[6],6.000000


In [14]:
 df.groupby('a')['b'].apply(list).reset_index(name='new')

,a,new
0,A,"[1, 2]"
1,B,"[5, 5, 4]"
2,C,[6]
